# Aim of this notebook

- How to analyse tweets
- How to do sentiment analysis
- How to make a wordcloud

# About the data

Dataset consists of tweets on Trump.


# About this Notebook

- <a href =#1> 1. Importing packages and libraries  </a>
- <a href =#2> 2. Reading the data </a>
- <a href =#3> 3. Handling null values </a>
- <a href =#4> 4. Exploratory data analysis </a>
- <a href =#5> 5. Cleaning the data </a>
- <a href =#6> 6. Applying data cleaning steps to data</a>
- <a href =#7> 7. Finding the most Common words in our Text</a>
- <a href =#8> 8.  Finding Most common words Sentiments Wise  </a>
    - <a href =#8.1> 8.1.  Finding the tweet sentiment</a>
    - <a href =#8.2> 8.2. Finding common words for positive sentiment tweets </a>
    - <a href =#8.3> 8.3. Finding the common words for negative sentiment tweets  </a>
    - <a href =#8.4> 8.4. Finding the common words for neutral sentiment tweets </a>
- <a href =#9> 9. Number of Unique Words in tweets of each type of sentiment  </a>
    - <a href =#9.1> 9.1. Number of unique words in tweets with positive sentiment </a>
    - <a href =#9.2> 9.2. Number of unique words in tweets with negative sentimen </a>
    - <a href =#9.3> 9.3. Number of unique words in tweets with neutral sentimen </a>
- <a href =#10> 10. Wordclouds  </a>
    - <a href =#10.1> 10.1.  WordCloud for neutral sentiment</a>
    - <a href =#10.2> 10.2.  WordCloud for positive sentiment</a>
    - <a href =#10.3> 10.3.  WordCloud for negative sentiment</a>

<br><br>
In case you are just starting with NLP here is a guide to Approach almost any NLP Problem by Grandmaster @Abhishek Thakur
https://www.slideshare.net/abhishekkrthakur/approaching-almost-any-nlp-problem


<b> This kernel is a work in Progress,and I will keep on updating it as I learn more and more</b>

**<span style="color:Red">If you find this kernel useful, Please Upvote it , it motivates me to write more Quality content**

<div id=1> <h2>  1. Importing packages and libraries  </h2> </div> 

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Below is a helper Function which generates random colors which can be used to give different colors to your plots.Feel free to use it**

In [ ]:
def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating the number of colours which are going to be generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

<div id=2> <h2> 2. Reading the Data  </h2></div> 

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv('/kaggle/input/trump-tweets/trumptweets.csv')


In [ ]:
train

In [ ]:
print(train.shape)


So We have 27486 tweets in the train set and 3535 tweets in the test set

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

Geo column has all values as nulls, hashtags have lot of null values, mentions also have lot of null values.

<div id=3> <h2> 3. Handling null values </h2> </div>

In [ ]:
train['hashtags'].unique()

In [ ]:
train['mentions'].unique()

1. hashtags: This column does not have something very significant for us to analyse. We have mostly words like #ixzz4 etc which make no sense and we wont be able to analyse their sentiment. So we can get rid of this column
2. mentions  too does not have anything siginificant for us to analyse. Hence we can do away with that too. Let us drop columns which dont have any siginificantly useful information.
3. geo anyways does not have any value, all values are null
4. mentions usually mention another person and we wont really get any sentiment by analysing that

In [ ]:
train=train.drop(['link','mentions','hashtags','geo'], axis=1)

<div id=4> <h2> 4. EDA  </h2></div>

In [ ]:
train.head()

Lets look at the distribution of reviews in the train set

<div id=5> <h2>5. Cleaning the data </h2> </div>

Let's first clean the data, remove stopwords etc and perform basic pre-processing

**Removing weird spaces**

In [ ]:
def remove_spaces(text):
    text=text.strip()
    text=text.split()
    return ' '.join(text)
    

<b> Spelling Correction </b>

In [ ]:
def edits1(word):
    letters='abcdefghijklmnopqrstuvwxyz'
    splits=[(word[:i], word[i:]) for i in range(len(word)+1)]
    deletes=[L+R[1:] for L,R in splits if R]
    transposes=[L+R[1] +R[0] + R[2:] for L,R in splits if len(R)>1]
    replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
    inserts = [L+c+ R for L,R in splits for c in letters]
    return set(deletes+transposes+replaces+inserts)
def edits2(word):
    return(e2 for e1 in edits1(word) for e2 in edits1(e1))
            

<b> Contraction </b>

In [ ]:
contraction = {'cause':'because',
              'aint': 'am not',
              'aren\'t': 'are not'}

def mapping_replacer(x,dic):
    for words in dic.keys():
        if ' ' + words + ' ' in x:
            x=x.replace(' '+ words +' ' ,' '+dic[words]+' ' )
    return x

    

<b> Stemming, lemmetisation and tokenisation
</b>

In [ ]:
import nltk
nltk.download('punkit')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.LancasterStemmer
ls = LancasterStemmer()
lem = WordNetLemmatizer()
def lexicon_normalization(text):
    words = word_tokenize(text) 
    
    
    # 1- Stemming
    words_stem = [ls.stem(w) for w in words]
    
    # 2- Lemmatization
    words_lem = [lem.lemmatize(w) for w in words_stem]
    return words_lem

    

<b> Handling emojis </b>

In [ ]:
import emoji
import re 
#from emot.emo_unicode import UNICODE_EMO
def convert_emojis(text):
    for emot in emoji.UNICODE_EMOJI:
        text = re.sub(r'('+emot+')', "_".join(emoji.UNICODE_EMOJI[emot].replace(",","").replace(":","").split()), text)
    return text

<b>Removing links, brackets, numbers, punctuations etc. </b>


In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\'','', text)
    
    return text

<b> Handling stopwords </b>

In [ ]:
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text

<b> Tokenisation </b>

In [ ]:
def tokenise(text):
    words = word_tokenize(text) 
    return words


<div id = 6> <h2> 6. Applying data cleaning steps to data </h2> </div>

Cleaning Regex Expressions from data

In [ ]:
import re
train['content'] = train['content'].map(lambda x: re.sub(r'\W+', ' ', x))
train['content'] = train['content'].replace(r'\W+', ' ', regex=True)


In [ ]:
train.head()

In [ ]:
train['content']=train['content'].apply(lambda x: mapping_replacer(x, contraction))

In [ ]:
train['content'] = train['content'].apply(lambda x:clean_text(x))

In [ ]:
train['content']=train['content'].apply(lambda x: remove_stopword(x))


In [ ]:
train['content']=train['content'].apply(lambda x: lexicon_normalization(x))

In [ ]:
train.head()

<div id=7><h2>7. Finding the most Common words in our Text </h2></div>

In [ ]:
top = Counter([item for sublist in train['content'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
blacklist = ['http','https','www','com', 'ev','u','ly','pic','would']

def remove_words(text):
    text = [i for i in text if (i not in blacklist)]
    return text

In [ ]:
#train['content']=remove_words(train['content'])

train['content'] = train['content'].apply(lambda x: [i for i in x if i not in blacklist])

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
top = Counter([item for sublist in train['content'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

<div id = 8><h2>8. Finding Most common words Sentiments Wise</h2></div>

Let's look at the most common words in different sentiments

<div id=8.1><h3>8.1 Finding the tweet sentiment </div></h3>

In [ ]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator

from textblob import TextBlob

def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(tweet) 
    
    # set sentiment 
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [ ]:
train['sentiment']=train['content'].apply(lambda x: get_tweet_sentiment(' '.join(x)))

In [ ]:
train.head()

In [ ]:
Positive_sent = train[train['sentiment']=='positive']
Negative_sent = train[train['sentiment']=='negative']
Neutral_sent = train[train['sentiment']=='neutral']

<b> Overall tweet sentiment </b>

In [ ]:
print('Number of tweets with positive sentiment', Positive_sent['sentiment'].count())
print('Number of tweets with negative sentiment', Negative_sent['sentiment'].count())
print('Number of tweets with neutral sentiment', Neutral_sent['sentiment'].count())


<b><i>So we see that overall the tweets are neutral in nature, followed by positive sentiment for the time in which they are analysed.</i></b>

<div id=8.2><h3>8.2 Finding common words for positive sentiment tweets</div></h3>

In [ ]:
#MosT common positive words
top = Counter([item for sublist in Positive_sent['content'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Greens')

Since we have words like 's', 'gre' which do not really mean anything, we will remove them and find the most common words

In [ ]:
import numpy as np
top = Counter([item for sublist in Positive_sent['content'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(23))
temp_positive.columns = ['Common_words','count']
temp_positive['Common_words'] = temp_positive['Common_words'].map(lambda x: re.sub(r'\W+', '', x))
temp_positive['Common_words'] = temp_positive['Common_words'].replace(r'\W+', '', regex=True)
temp_positive['Common_words'] = temp_positive['Common_words'].apply(lambda x:remove_spaces(x))
temp_positive=temp_positive[~temp_positive['Common_words'].isin(['s','gre','“',' * '])] #new line removing meaningless words
mask1 = temp_positive.Common_words.str.contains('[a-zA-Z]')
mask2 = temp_positive.Common_words.notna()
temp_positive = temp_positive[mask1 | mask2]
temp_positive.Common_words =  temp_positive.Common_words.str.replace(r"\s+", "").replace("", np.NaN)
temp_positive=temp_positive.dropna()


temp_positive.style.background_gradient(cmap='Greens')


In [ ]:
fig = px.bar(temp_positive, x="count", y="Common_words", title='Most Commmon Words in Positive Sentiment tweets', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

<div id=8.3><h3>8.3 Finding the common words for negative sentiment tweets </div></h3>

In [ ]:
#MosT common negative words
top = Counter([item for sublist in Negative_sent['content'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common negative words
top = Counter([item for sublist in Negative_sent['content'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(22))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']

#Data cleaning
temp_negative['Common_words'] = temp_negative['Common_words'].map(lambda x: re.sub(r'\W+', '', x))
temp_negative['Common_words'] = temp_negative['Common_words'].replace(r'\W+', '', regex=True)
temp_negative=temp_negative[~temp_negative['Common_words'].isin(['s','t'])] #new line removing meaningless words from above
#mask1 = temp_negative.Common_words.str.contains('[a-zA-Z]')
#mask2 = temp_negative.Common_words.notna()
#temp_negative = temp_negative[mask1 | mask2]

temp_negative.Common_words =  temp_negative.Common_words.replace("", np.nan)
temp_negative = temp_negative.dropna(subset=['Common_words'])

temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
fig = px.treemap(temp_negative, path=['Common_words'], values='count',title='Tree Of Most Common Words in Negative Tweets')
fig.show()

<div id=8.4><h3>8.4 Finding the common words for neutral sentiment tweets </div></h3>

In [ ]:
#MosT common Neutral words
top = Counter([item for sublist in Neutral_sent['content'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

In [ ]:

top = Counter([item for sublist in Neutral_sent['content'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']

#Data cleaning
temp_neutral['Common_words'] = temp_neutral['Common_words'].map(lambda x: re.sub(r'\W+', '', x))
temp_neutral['Common_words'] = temp_neutral['Common_words'].replace(r'\W+', '', regex=True)
temp_neutral=temp_neutral[~temp_neutral['Common_words'].isin(['s'])] #new line removing meaningless words from above

temp_neutral.Common_words =  temp_neutral.Common_words.replace("", np.nan)
temp_neutral = temp_neutral.dropna(subset=['Common_words'])

temp_neutral.style.background_gradient(cmap='Reds')

In [ ]:
fig = px.bar(temp_neutral, x="count", y="Common_words", title='Most Commmon Neutral Words', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
fig = px.treemap(temp_neutral, path=['Common_words'], values='count',title='Tree Of Most Common Neutral Words')
fig.show()

<div id=9><h2>9. Number of Unique Words in tweets of each type of sentiment </h2> </div>

We will look at number of unique words in each type of tweet with different sentiments:
* Positive
* Negative
* Neutral

In [ ]:
raw_text = [word for word_list in train['content'] for word in word_list]

In [ ]:
def words_unique(sentiment,numwords,raw_words):
    '''
    Input:
        segment - Segment category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result; 
        raw_words - list  for item in train_data[train_data.segments == segments]['temp_list1']:
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in train[train.sentiment != sentiment]['content']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in train[train.sentiment == sentiment]['content']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words

  <div id=9.1><h2>9.1. Number of unique words in tweets with positive sentiment  </h2> </div>

In [ ]:
Unique_Positive= words_unique('positive', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
Unique_Positive

In [ ]:
fig = px.treemap(Unique_Positive, path=['words'], values='count',title='Tree Of Unique Words in Positive sentiment tweets')
fig.show()

In [ ]:
import 
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Positive['count'], labels=Unique_Positive.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique words in Positive sentiment tweets')
plt.show()

  <div id=9.2><h2>9.2. Number of unique words in tweets with negative sentiment  </h2> </div>

In [ ]:
Unique_Negative= words_unique('negative', 10, raw_text)
print("The top 10 unique words in Negative Tweets are:")
Unique_Negative.style.background_gradient(cmap='Reds')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.rcParams['text.color'] = 'black'
plt.pie(Unique_Negative['count'], labels=Unique_Negative.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique words in Negative sentiment tweets')
plt.show()

  <div id=9.3><h2>9.3. Number of unique words in tweets with neutral sentiments  </h2> </div>

In [ ]:
Unique_Neutral= words_unique('neutral', 10, raw_text)
print("The top 10 unique words in Neutral Tweets are:")
Unique_Neutral.style.background_gradient(cmap='Oranges')

Since the count for positive, negative and neutral sentiments is so less, there is no use of creating word clouds with unique words. So we will create a wordcloud with all the words combined.

In [ ]:
#Data cleaning
Unique_Neutral= words_unique('neutral', 14, raw_text)
Unique_Neutral['words'] = Unique_Neutral['words'].map(lambda x: re.sub(r'\W+', '', x))
Unique_Neutral['words'] = Unique_Neutral['words'].replace(r'\W+', '', regex=True)
Unique_Neutral['words']=Unique_Neutral[~Unique_Neutral['words'].isin(['به','را','ایران','و'])] #new line removing meaningless words from above

Unique_Neutral['words'] =  Unique_Neutral['words'].replace("", np.nan)
Unique_Neutral= Unique_Neutral.dropna(subset=['words'])

Unique_Neutral.style.background_gradient(cmap='Reds')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Neutral['count'], labels=Unique_Neutral.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique words in Neutral sentiment tweets')
plt.show()

**By Looking at the Unique Words of each sentiment,we now have much more clarity about the data**

We shall now proceed to create word clouds, but we shall make word cloud of all words and not just existing words since unique words are very less in number for many sentiments (10,7,9 etc.) , so an effective word cloud can be only made by considering all the words in different sentiment tweets

<div id=10><h2>10. WordClouds</h2> </div>


We will be building three types of wordclouds:

* WordCloud of Neutral Tweets
* WordCloud of Positive Tweets
* WordCloud of Negative Tweets

In [ ]:
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), color = 'white',
                   title = None, title_size=40, image_color=False):
    
    wordcloud = WordCloud(background_color=color,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
d = '/kaggle/input/trump-tweets/'

<div id=10><h2>10.1. WordCloud for neutral sentiment</h2> </div>

In [ ]:
Neutral_sent

In [ ]:
pos_mask = np.array(Image.open('/kaggle/input/tweet-mask/tweet_mask.png'))
plot_wordcloud(Neutral_sent.content,mask=pos_mask,color='white',max_font_size=100,title_size=30,title="WordCloud of Neutral Tweets")

<div id=10.2><h2>10.2 WordCloud for positive sentiment</h2> </div>


In [ ]:
plot_wordcloud(Positive_sent.content,mask=pos_mask,title="Word Cloud Of Positive tweets",title_size=30)

<div id=10.3><h2>10.3 WordCloud for Tweets with negative sentiment</h2> </div>

In [ ]:
plot_wordcloud(Negative_sent.content,mask=pos_mask,title="Word Cloud of Negative Tweets",color='white',title_size=30)

<h4>If you liked my kernel, please be kind to upvote it :). It will help my efforts :)</h4>